Importing Libraries

In [45]:
import sys
import os
import numpy as np
import renders as rs
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from IPython.display import display # Allows the use of display() for DataFrames
from sklearn.externals import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFECV
from sklearn.cross_validation import StratifiedKFold
from sklearn.svm import SVC
# Show matplotlib plots inline (nicely formatted in the notebook)
%matplotlib inline
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/match_stats.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_model.py')
sys.path.append('/anaconda/envs/stats/lib/python3.5/site-packages')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/model_libs.py')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats/form_data.py')
sys.path.append('/Users/senzari/Machine_Learning/stats')
sys.path.append('/Users/senzari/Machine_Learning/stats/src/stats')
# print(sys.path)
os.chdir('/Users/senzari/Machine_Learning/stats/src')
#print(os.getcwd())

# Variables
round_number = 26
target_col = 'points'
ignore_cols = ['match_id', 'team_id', 'team_name', 'opp_id', 'opp_name', 'scheduled']
testing = False

from stats import form_data, match_stats, model_libs, form_model, predict_matches

Initially use features just in database, ran PCA and tried to enhance the features that had the most influence...

Data essentially has 2 variances.  Home vs Away stats and the Extended Features of Current Team, Opponent, Current Teams Previous Opponents, and Opponents Opponent

In [38]:
data_csv = 'round' + str(round_number) + '.csv'

if testing:
    raw_data = form_data.run_data()
    raw_data.to_csv(data_csv)
    print("Raw Data Saved to CSV")
else:
    #Reading in a CSV adds the first index column
    raw_data = pd.read_csv(data_csv)
    raw_data = raw_data.drop(raw_data.columns[[0]], axis=1)

pd.set_option("display.max_columns", 85)
print('Data Loaded...')

Data Loaded...


In [116]:
print(raw_data.shape)
#display(raw_data.head())

(480, 82)


In [117]:
models = ['svc', 'gmm', 'kmeans', 'gnb']

def train_models(round_num, X, y):
    
    if os.path.isdir("/models/" + str(round_num)):
        print('Making New Directory for the Round')
        os.chdir('/Users/senzari/Machine_Learning/stats/src/models')
        os.makedirs(str(round_num))
        os.chdir('/Users/senzari/Machine_Learning/stats/src')
    
    for i in models:
        
        model_round = 'models/' + str(round_num) + '/' + str(i) + '_round_' + str(round_num) + '.pk1'
        
        if i == 'svc':
            svc = form_model.build_model(X, y, i)
            joblib.dump(svc, model_round)
        elif i == 'gmm':
            gmm = form_model.build_model(X, y, i)
            joblib.dump(gmm, model_round)
        elif i == 'kmeans':
            kmeans = form_model.build_model(X, y, i)
            joblib.dump(kmeans, model_round)
        elif i == 'gnb':
            gnb = form_model.build_model(X, y, i)
            joblib.dump(gnb, model_round)
        
    return svc, gmm, kmeans, gnb
        
        
def load_models(round_num):
    
    for i in models:
        model_round = 'models/' + str(round_num) + '/' + str(i) + '_round_' + str(round_num) + '.pk1'
        if i == 'svc':
             svc= joblib.load(model_round)
        elif i == 'gmm':
            gmm = joblib.load(model_round)
        elif i == 'kmeans':
            kmeans = joblib.load(model_round)
        elif i == 'gnb':
            gnb = joblib.load(model_round)
        
        print("Success :: Loaded - " + str(i))
        
    return svc, gmm, kmeans, gnb


# svc, gmm, kmeans, gnb = train_models(round_number, X, y)
# svc, gmm, kmeans, gnb = load_models(round_number)

Find all matches

In [ ]:
upcoming_matches, match_details = predict_matches.get_upcoming_matches()
print(upcoming_matches)

In [ ]:
for i in models:
    if i == 'svc':
        svc_preds, upcoming_data = predict_matches.predictions(upcoming_matches, match_details, svc)
        print(svc_preds)
    elif i == 'gmm':
        gmm_preds, upcoming_data = predict_matches.predictions(upcoming_matches, match_details, gmm)
        print(gmm_preds)
    elif i == 'kmeans':
        kmeans_preds, upcoming_data = predict_matches.predictions(upcoming_matches, match_details, kmeans)
        print(kmeans_preds)
    elif i == 'gnb':
        gnb_preds, upcoming_data = predict_matches.predictions(upcoming_matches, match_details, gnb)
        print(gnb_preds)

Adds list of predictions to the upcoming matches and puts them in a CSV

In [ ]:
columns = ['team_name', 'opp_name', 'scheduled']
# Remove all columns except the ones above
upcoming_matches = upcoming_data[columns]
# Add predictions to the end of that DF
results = pd.DataFrame({'SVC': svc_preds, 'GMM': gmm_preds, 'K-Means': kmeans_preds, 'GNB': gnb_preds})
upcoming_matches = upcoming_matches.join(results)
reordered_matches = pd.DataFrame([])

for rows in upcoming_matches.iterrows():
    for i in upcoming_matches['team_name']:
        if rows[1]['opp_name'] == i:
            reordered_matches = reordered_matches.append(rows[1])
            reordered_matches = reordered_matches.append(upcoming_matches[upcoming_matches['team_name'].isin([i])])

reordered_matches = reordered_matches.drop_duplicates() 
columns = ['scheduled', 'team_name', 'opp_name', 'SVC', 'K-Means', 'GMM', 'GNB']
reordered_matches = reordered_matches[columns]
# upcoming_matches = upcoming_matches[(upcoming_matches['scheduled'] < '2016-08-26')]
reordered_matches.to_csv('predictions_' + str(round_number) + '.csv')

Tune Models
Doesn't really do anything now, need to ask some questions to get better.  Maybe removing some features will help.

In [ ]:
for i in models:
    form_model.build_tuned_model(X, y, i)

Trying to remove some useless features

In [ ]:
print(raw_data.shape)
reduced_data = form_data.variance_features(raw_data)
print(reduced_data.shape)

In [ ]:
svc = SVC(kernel="linear")
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(y, 2),
              scoring='accuracy')
rfecv.fit(X, y)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

# Need to normalize the data
"""norm_data = X.copy(deep=True)
norm_data = norm_data.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
X_new = SelectKBest(chi2, k=50).fit_transform(norm_data, y)
print(X.shape)"""

In [ ]:
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.feature_selection import RFE

# Create the RFE object and rank each pixel
svc = SVC(kernel="linear", C=1)
rfe = RFE(estimator=svc, n_features_to_select=1, step=1)
rfe.fit(X, y)
print(rfe.ranking_)


In [ ]:
ranked_features = pd.DataFrame(index=range(448))
for a in range(len(rfe.ranking_)):
    for i in range(len(rfe.ranking_)):
        if rfe.ranking_[i] == a+1:
            ranked_features = ranked_features.join(X.ix[:,i])
            
display(ranked_features.describe())

Let's try to reduce the number of features to at least 25 although optimal seems likely around ~8 features

In [ ]:
for x in xrange(5, 25):
    print('X :: {}'.format(x))
    svc, gmm, kmeans, gnb = train_models(round_number, ranked_features.iloc[:, 1:x], y)
    print(' ============================ ')
    

try PCA on original training data 

In [ ]:
pca = PCA(n_components=5)
pca.fit(X)
# Generate PCA results plot
pca_results = rs.pca_results(X, pca)

In [ ]:
# TODO: Apply a PCA transformation the good data
reduced_data = pca.transform(X)
# display(pd.DataFrame(np.round(reduced_data, 4), columns = ['D1', 'D2', 'D3', 'D4', 'D5']))
svc, gmm, kmeans, gnb = train_models(round_number, reduced_data, y)

In [ ]:
Try PCA on ranked features 

In [ ]:
svc, gmm, kmeans, gnb = train_models(round_number, reduced_data, y)

In [ ]:
pca = PCA(n_components=5)
pca.fit(ranked_features.iloc[:, 1:23]) # 23 was just the highest value when testing
# Generate PCA results plot
pca_results = rs.pca_results(ranked_features.iloc[:, 1:23], pca)

In [ ]:
# TODO: Apply a PCA transformation the good data
reduced_data = pca.transform(ranked_features.iloc[:, 1:23])
# display(pd.DataFrame(np.round(reduced_data, 4), columns = ['D1', 'D2', 'D3', 'D4', 'D5']))
svc, gmm, kmeans, gnb = train_models(round_number, reduced_data, y)

Let's try removing ties similar to the other project

In [ ]:
target_col = 'points'
ignore_cols = ['match_id', 'team_id', 'team_name', 'opp_id', 'opp_name', 'scheduled']

filtered_data = raw_data[raw_data.points != 1]
fd = model_libs._clone_and_drop(filtered_data, ignore_cols)
(f_y, f_X) = model_libs._extract_target(fd, target_col)
print(f_X.shape)

In [ ]:
svc, gmm, kmeans, gnb = train_models(round_number, f_X, f_y)

In [121]:
# ha = home-away
standard_cols = ['games_played', 'is_home', 'avg_points', 'goals_for', 'goals_against', 'avg_goals', 'margin', 'goal_diff',
                'win_percentage', 'sos', 'opp_avg_points', 'opp_avg_goals', 'opp_margin', 'opp_goal_diff', 'opp_win_percentage',
                'opp_opp_record', 'points']
# 17 each
home_cols = ['current_team_home_possession', 'current_team_home_attacks', 'current_team_home_dangerous_attacks', 'current_team_home_yellow_card',
            'current_team_home_corner_kicks', 'current_team_home_shots_on_target', 'current_team_home_shots_total', 'current_team_home_ball_safe',
            'current_team_home_played', 'current_opp_away_attacks', 'current_opp_away_dangerous_attacks', 'current_opp_away_yellow_card',
            'current_opp_away_corner_kicks', 'current_opp_away_shots_on_target', 'current_opp_away_shots_total', 'current_opp_away_ball_safe',
            'current_opp_away_played']
away_cols = ['current_team_away_possession', 'current_team_away_attacks', 'current_team_away_dangerous_attacks', 'current_team_away_yellow_card', 
            'current_team_away_corner_kicks', 'current_team_away_shots_on_target', 'current_team_away_shots_total', 'current_team_away_ball_safe', 
            'current_team_away_played', 'current_opp_home_attacks', 'current_opp_home_dangerous_attacks', 'current_opp_home_yellow_card', 'current_opp_home_corner_kicks', 
             'current_opp_home_shots_on_target', 'current_opp_home_shots_total', 'current_opp_home_ball_safe', 'current_opp_home_played']

def rename_column(label):
    if label.count('_home_') > 0:
        new_name = label.replace('_home_', '_')
    elif label.count('_away_') > 0:
        new_name = label.replace('_away_', '_')
    return new_name

# Creates new column list for merged columns
merged_cols = []

for cols in home_cols:
    merged_cols.append(rename_column(cols))
    
# Creates a Home and Away Table with Standard and Home/Away Columns.  Will evenutally merge together
ha_data = model_libs._clone_and_drop(raw_data, ignore_cols)
home_data = ha_data.loc[ha_data.loc[:, 'is_home'] == 1, standard_cols + home_cols]
away_data = ha_data.loc[ha_data.loc[:, 'is_home'] == 0, standard_cols + away_cols]

#display(away_data.describe())
#display(home_data.describe())

# Appends the Home Table with the Away Table
ha_data = home_data.append(away_data)

def pick_column(home, away):
    if np.isnan(home):
        return away
    elif np.isnan(away):
        return home

# Combine Home/Away Columns into one for Current Team and Opponent.  Cuts down on Features
# Adds from whatever column that doesn't have Nan
for c in range(len(merged_cols)):
    ha_data[merged_cols[c]] = ha_data.apply(lambda row: pick_column(row[home_cols[c]], row[away_cols[c]]), axis=1 )

ha_data = ha_data.drop(home_cols + away_cols, axis=1)
    
display(ha_data.describe())

print(ha_data.shape)
#(y, X) = model_libs._extract_target(home_away_data, target_col)

,avg_goals,avg_points,games_played,goal_diff,goals_against,goals_for,is_home,margin,opp_avg_goals,opp_avg_points,opp_goal_diff,opp_margin,opp_opp_record,opp_win_percentage,points,sos,win_percentage,current_team_possession,current_team_attacks,current_team_dangerous_attacks,current_team_yellow_card,current_team_corner_kicks,current_team_shots_on_target,current_team_shots_total,current_team_ball_safe,current_team_played,current_opp_attacks,current_opp_dangerous_attacks,current_opp_yellow_card,current_opp_corner_kicks,current_opp_shots_on_target,current_opp_shots_total,current_opp_ball_safe,current_opp_played
count,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000,480.000000
mean,1.018750,0.977083,12.393750,-0.041667,1.364583,1.387500,0.500000,-0.427083,1.018750,0.977083,-0.041667,-0.427083,0.500201,0.499875,1.333333,1.166982,0.499875,50.004392,100.841481,50.969118,1.659965,4.988794,4.889909,9.345730,93.778691,5.995833,100.841481,50.969118,1.659965,4.988794,4.889909,9.345730,93.778691,5.995833
std,0.506392,0.557260,6.922262,4.373761,1.142460,1.150406,0.500522,0.692109,0.506392,0.557260,4.373761,0.692109,0.164743,0.164815,1.248520,0.304611,0.164815,3.825790,12.731148,8.455986,0.597197,1.656228,1.292097,2.071212,10.554401,3.545703,12.731148,8.455986,0.597197,1.656228,1.292097,2.071212,10.554401,3.545703
min,0.000000,0.000000,1.000000,-20.000000,0.000000,0.000000,0.000000,-3.000000,0.000000,0.000000,-20.000000,-3.000000,0.000000,0.000000,0.000000,0.185185,0.000000,34.000000,65.000000,28.500000,0.000000,0.000000,0.000000,3.000000,45.000000,0.000000,65.000000,28.500000,0.000000,0.000000,0.000000,3.000000,45.000000,0.000000
25%,1.000000,1.000000,6.000000,-3.000000,0.000000,0.000000,0.000000,-1.000000,1.000000,1.000000,-3.000000,-1.000000,0.407407,0.408670,0.000000,0.997418,0.408670,47.500000,93.000000,45.650000,1.296429,3.833333,4.000000,8.000000,88.750000,3.000000,93.000000,45.650000,1.296429,3.833333,4.000000,8.000000,88.750000,3.000000
50%,1.000000,1.000000,12.500000,0.000000,1.000000,1.000000,0.500000,0.000000,1.000000,1.000000,0.000000,0.000000,0.500000,0.500000,1.000000,1.152499,0.500000,50.000000,98.850000,50.428571,1.600000,5.000000,4.857143,9.045455,93.707143,6.000000,98.850000,50.428571,1.600000,5.000000,4.857143,9.045455,93.707143,6.000000
75%,1.000000,1.000000,18.000000,2.000000,2.000000,2.000000,1.000000,0.000000,1.000000,1.000000,2.000000,0.000000,0.590909,0.588612,3.000000,1.336093,0.588612,52.200000,109.343750,55.500000,2.000000,6.000000,5.666667,10.666667,99.666667,9.000000,109.343750,55.500000,2.000000,6.000000,5.666667,10.666667,99.666667,9.000000
max,4.000000,3.000000,26.000000,13.000000,7.000000,7.000000,1.000000,3.000000,4.000000,3.000000,13.000000,3.000000,1.000000,1.000000,3.000000,2.166667,1.000000,66.000000,143.000000,78.400000,5.000000,12.000000,9.000000,16.000000,128.000000,14.000000,143.000000,78.400000,5.000000,12.000000,9.000000,16.000000,128.000000,14.000000


(480, 34)


In [122]:
(y, X) = model_libs._extract_target(ha_data, target_col)
print(X.shape)

(480, 33)


In [123]:
svc, gmm, kmeans, gnb = train_models(round_number, X, y)

Training SVC Modeling


//anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
//anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_

F1 score for training set: 0.580827447809
F1 score for test set: 0.462373060265
Finished SVC Modeling
Training GMM Modeling
F1 score for training set: 0.130283816425
F1 score for test set: 0.166935483871
('Scores :', [0.10918065522018523])
Finished GMM Modeling
Training K-Means Modeling
F1 score for training set: 0.226875178542
F1 score for test set: 0.268656501726
Finished SVC Modeling
Finished K-Means Modeling
Training Gaussian NB Modeling
F1 score for training set: 0.523462734498
F1 score for test set: 0.464802014518
Finished Gaussian NB Modeling


//anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)
//anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_

Now testing table with standard and extended features

In [124]:
# ef = extended features
ef_data = model_libs._clone_and_drop(raw_data, ignore_cols)
print(ef_data.shape)
ef_data = ef_data.drop(home_cols + away_cols, axis=1)
print(ef_data.shape)

(480, 76)
(480, 42)


In [ ]:
(y, X) = model_libs._extract_target(ef_data, target_col)
svc, gmm, kmeans, gnb = train_models(round_number, X, y)

Training SVC Modeling
